<a href="https://colab.research.google.com/github/msrjdin/TransformerUseCases/blob/master/Text%20Classification/xlm-mlm-xnli15-1024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers

     |████████████████████████████████| 890kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 23.1MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
     |████████████████████████████████| 1.1MB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4d57065f3a0b81038fec0292fc962572f51c22f004b341c215f09265bad8db10
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import pandas as pd

In [6]:
df = pd.read_csv('Data.csv')

In [7]:
df.dropna(inplace=True)

In [8]:
df.shape

(67730, 5)

In [9]:
df.head()

,asin,Rating5,title,body1,price
0,B0009N5L7K,0,Stupid phone,BUY SERVICE,49.95
1,B0009N5L7K,0,Exellent Service,nextel nearly year started time last year Moto...,49.95
2,B0009N5L7K,1,I love it,"got say easy use, hear person talking fine pro...",49.95
3,B0009N5L7K,0,Phones locked,1 star phones locked pay additional fees unlock,49.95
4,B0009N5L7K,1,Excellent product,product good. used cell phone one projects wor...,49.95


In [10]:
df.rename(columns = {'body1':'text', 'Rating5':'5star'}, inplace=True)

In [11]:
df.reset_index(drop= True, inplace=True)

In [12]:
df.head()

,asin,5star,title,text,price
0,B0009N5L7K,0,Stupid phone,BUY SERVICE,49.95
1,B0009N5L7K,0,Exellent Service,nextel nearly year started time last year Moto...,49.95
2,B0009N5L7K,1,I love it,"got say easy use, hear person talking fine pro...",49.95
3,B0009N5L7K,0,Phones locked,1 star phones locked pay additional fees unlock,49.95
4,B0009N5L7K,1,Excellent product,product good. used cell phone one projects wor...,49.95


In [13]:
from sklearn.model_selection import train_test_split
X= df['text'].copy()
y= df['5star']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [18]:
#Setting the model_name runs the model required
model_name = "xlm-mlm-xnli15-1024"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at xlm-mlm-xnli15-1024 were not used when initializing XLMForSequenceClassification: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing XLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMForSequenceClassification were not initialized from the model checkpoint at xlm-mlm-xnli15-1024 and are newly initialized: ['transformer.position_ids', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [19]:
tr_enc = tokenizer(list(X_train.values), padding=True, truncation=True, max_length=100)
tst_enc = tokenizer(list(X_test.values),  padding=True, truncation=True,  max_length=100)

In [20]:
import torch

class buildingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = buildingDataset(tr_enc, list(y_train.values))
val_dataset = buildingDataset(tst_enc, list(y_test.values))

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

trainer = Trainer(
    model=pt_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics
)


In [23]:
trainer.train()

{'loss': 3.2940250396728517, 'learning_rate': 5e-06, 'epoch': 0.0033738191632928477, 'step': 10}
{'loss': 1.8317577362060546, 'learning_rate': 1e-05, 'epoch': 0.006747638326585695, 'step': 20}
{'loss': 2.14752197265625, 'learning_rate': 1.5e-05, 'epoch': 0.010121457489878543, 'step': 30}
{'loss': 2.241743469238281, 'learning_rate': 2e-05, 'epoch': 0.01349527665317139, 'step': 40}
{'loss': 1.5294525146484375, 'learning_rate': 2.5e-05, 'epoch': 0.016869095816464237, 'step': 50}
{'loss': 2.0808258056640625, 'learning_rate': 3e-05, 'epoch': 0.020242914979757085, 'step': 60}
{'loss': 2.47298583984375, 'learning_rate': 3.5e-05, 'epoch': 0.023616734143049933, 'step': 70}
{'loss': 1.861151123046875, 'learning_rate': 4e-05, 'epoch': 0.02699055330634278, 'step': 80}
{'loss': 2.015458679199219, 'learning_rate': 4.5e-05, 'epoch': 0.030364372469635626, 'step': 90}
{'loss': 1.2470626831054688, 'learning_rate': 5e-05, 'epoch': 0.033738191632928474, 'step': 100}
{'loss': 1.2814117431640626, 'learning_

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 0.819000244140625, 'learning_rate': 4.648249828414551e-05, 'epoch': 0.1720647773279352, 'step': 510}
{'loss': 0.746435546875, 'learning_rate': 4.639670555936857e-05, 'epoch': 0.17543859649122806, 'step': 520}
{'loss': 0.762139892578125, 'learning_rate': 4.631091283459163e-05, 'epoch': 0.1788124156545209, 'step': 530}
{'loss': 0.7431884765625, 'learning_rate': 4.622512010981469e-05, 'epoch': 0.18218623481781376, 'step': 540}
{'loss': 0.7203857421875, 'learning_rate': 4.613932738503775e-05, 'epoch': 0.18556005398110662, 'step': 550}
{'loss': 0.7864501953125, 'learning_rate': 4.605353466026081e-05, 'epoch': 0.18893387314439947, 'step': 560}
{'loss': 0.79278564453125, 'learning_rate': 4.596774193548387e-05, 'epoch': 0.19230769230769232, 'step': 570}
{'loss': 0.811639404296875, 'learning_rate': 4.588194921070693e-05, 'epoch': 0.19568151147098514, 'step': 580}
{'loss': 0.75487060546875, 'learning_rate': 4.5796156485929995e-05, 'epoch': 0.199055330634278, 'step': 590}
{'loss': 0.8388

{'loss': 0.69931640625, 'learning_rate': 2.5377487989018532e-05, 'epoch': 1.0020242914979758, 'step': 2970}
{'loss': 0.663720703125, 'learning_rate': 2.5291695264241594e-05, 'epoch': 1.0053981106612686, 'step': 2980}
{'loss': 0.6349853515625, 'learning_rate': 2.5205902539464653e-05, 'epoch': 1.0087719298245614, 'step': 2990}
{'loss': 0.675537109375, 'learning_rate': 2.5120109814687715e-05, 'epoch': 1.0121457489878543, 'step': 3000}
{'loss': 0.685205078125, 'learning_rate': 2.5034317089910774e-05, 'epoch': 1.015519568151147, 'step': 3010}
{'loss': 0.6770263671875, 'learning_rate': 2.494852436513384e-05, 'epoch': 1.01889338731444, 'step': 3020}
{'loss': 0.7360595703125, 'learning_rate': 2.4862731640356898e-05, 'epoch': 1.0222672064777327, 'step': 3030}
{'loss': 0.7233154296875, 'learning_rate': 2.477693891557996e-05, 'epoch': 1.0256410256410255, 'step': 3040}
{'loss': 0.731640625, 'learning_rate': 2.4691146190803022e-05, 'epoch': 1.0290148448043186, 'step': 3050}
{'loss': 0.6726318359375

TrainOutput(global_step=5928, training_loss=0.7056701083575827)

In [24]:
trainer.evaluate()


{'eval_loss': 0.541080752072756, 'eval_accuracy': 0.7641616221270732, 'eval_f1': 0.7855544616486172, 'eval_precision': 0.7961719883889695, 'eval_recall': 0.7752163928634517, 'epoch': 2.0, 'step': 5928}


{'epoch': 2.0,
 'eval_accuracy': 0.7641616221270732,
 'eval_f1': 0.7855544616486172,
 'eval_loss': 0.541080752072756,
 'eval_precision': 0.7961719883889695,
 'eval_recall': 0.7752163928634517}